In [98]:
import pandas as pd
import numpy as np
import mysql.connector
import os
import time

In [99]:
# discover which candidates need data still
mydb = mysql.connector.connect(
        host="localhost",
        port=3308,
        user="root", 
        password="secret",
        database="ftvBackEnd"
        )
cursor = mydb.cursor()
query = """
SELECT
    cid
FROM 
    candidate
where 
    cycle = 2020 
    AND id not in  (
        SELECT candidate_id FROM `candidate-industry`
        ) 
"""
cursor.execute(query)
still_needs_data = cursor.fetchall()
print(len(still_needs_data))
mydb.close()


3350


In [100]:
url = ("https://www.opensecrets.org/api/?method=candIndustry&cid="
       "{cid}"
       "&cycle="
       "{cycle}"
       "&apikey="
       "{apikey}"
       "&output=json"
      )
cycle=2020
apikey="wouldn't you like to know"
json_list = []
no_resource_found = []
for candidate in still_needs_data:
    open_secret_request = requests.get(url.format(cid=candidate[0], cycle=cycle, apikey=apikey))
    time.sleep(1)
    print(open_secret_request)
    if open_secret_request.ok:
        json_list.append(open_secret_request.json())
    elif open_secret_request.status_code == 404:
        no_resource_found.append(candidate)
    else:
        break
    

<Response [400]>


In [101]:
def insert_new_industry(code, name, cursor):
    query = """
    INSERT IGNORE INTO 
        industry(`code`, name)
    VALUES
        (%s, %s)
    """
    cursor.execute(query, (code, name))

def populate_ftv(cid, cycle, last_updated, industry_array, cursor):

    clean_cid_cycle_query = """
    DELETE FROM 
        `candidate-industry` 
    WHERE
        candidate_id = (SELECT id FROM candidate WHERE cid=%s AND cycle=%s);
    """
    insert_funding_query = """
    INSERT INTO
        `candidate-industry`(candidate_id, industry_id, indivs, pacs, total, last_updated)
    VALUES
        (
        (SELECT id FROM candidate WHERE cid=%s AND cycle=%s),
        (SELECT id FROM industry WHERE `code`=%s),
        %s,
        %s,
        %s,
        %s
        )
    """
    cursor.execute(clean_cid_cycle_query, (cid, cycle))
    if len(industry_array) == 1:
        insert_new_industry(
            industry_array["@attributes"]["industry_code"],
            industry_array["@attributes"]["industry_name"],
            cursor)
        cursor.execute(insert_funding_query, (
            cid, 
            cycle,
            industry_array["@attributes"]["industry_code"],
            industry_array["@attributes"]["indivs"],
            industry_array["@attributes"]["pacs"],
            industry_array["@attributes"]["total"],
            last_updated
        ))
    else:
        for industry in industry_array:
            
            insert_new_industry(
                industry["@attributes"]["industry_code"],
                industry["@attributes"]["industry_name"],
                cursor)
            params = (
                cid, 
                cycle,
                industry["@attributes"]["industry_code"],
                industry["@attributes"]["indivs"],
                industry["@attributes"]["pacs"],
                industry["@attributes"]["total"],
                last_updated
            )
            cursor.execute(insert_funding_query, params)


mydb = mysql.connector.connect(
    host="localhost",
    port=3308,
    user="root", 
    password="secret",
    database="ftvBackEnd"
    )
    
cursor = mydb.cursor()
for candidate in json_list:
    cid = candidate['response']['industries']['@attributes']['cid']
    cycle = candidate['response']['industries']['@attributes']['cycle']
    last_updated = candidate['response']['industries']['@attributes']['last_updated']
    industries = candidate['response']['industries']['industry']
    populate_ftv(cid, cycle, last_updated, industries, cursor)
    mydb.commit()
    
mydb.close()

In [97]:
json_list

[{'response': {'industries': {'@attributes': {'cand_name': 'Dorothy Benford (D)',
     'cid': 'N00012192',
     'cycle': '2020',
     'origin': 'Center for Responsive Politics',
     'source': 'https://www.opensecrets.org/members-of-congress/industries?cid=N00012192&cycle=2020',
     'last_updated': '03/22/2021'},
    'industry': [{'@attributes': {'industry_code': 'N12',
       'industry_name': 'Misc Business',
       'indivs': '100',
       'pacs': '0',
       'total': '100'}},
     {'@attributes': {'industry_code': 'W04',
       'industry_name': 'Education',
       'indivs': '53',
       'pacs': '0',
       'total': '53'}},
     {'@attributes': {'industry_code': 'W06',
       'industry_name': 'Retired',
       'indivs': '49',
       'pacs': '0',
       'total': '49'}},
     {'@attributes': {'industry_code': 'F07',
       'industry_name': 'Securities & Investment',
       'indivs': '34',
       'pacs': '0',
       'total': '34'}},
     {'@attributes': {'industry_code': 'W02',
       '